# Native Server API

Apart from the OpenAI compatible API, the SGLang Runtime also provides its native server API. We introduce these following API:

- `/generate`
- `/update_weights`
- `/get_server_args`
- `/get_model_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/get_memory_pool_size`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch The Server

In [1]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)
import subprocess, json

server_process = execute_shell_command(
    """
    python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --port=30010
"""
)

wait_for_server("http://localhost:30010")

/home/chenyang/miniconda3/envs/AlphaMeemory/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
[2024-11-01 21:15:39] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, is_embedding=False, host='127.0.0.1', port=30010, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=68951524, constrained_json_whitespace_pattern=None, decode_log_interval=40, lo

## Generate

Used to generate completion from the model, similar to the `/v1/chat/completions` API in OpenAI.

In [3]:
# generate

import requests

url = "http://localhost:30010/generate"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer None"
}
data = {
    "text": "List 3 countries and their capitals."
}

response = requests.post(url, headers=headers, json=data)
print_highlight(response.text)

[2024-11-01 21:16:16 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 8, cache hit rate: 36.00%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-01 21:16:16 TP0] Decode batch. #running-req: 1, #token: 35, token usage: 0.00, gen throughput (token/s): 8.13, #queue-req: 0
[2024-11-01 21:16:16 TP0] Decode batch. #running-req: 1, #token: 75, token usage: 0.00, gen throughput (token/s): 213.64, #queue-req: 0
[2024-11-01 21:16:17 TP0] Decode batch. #running-req: 1, #token: 115, token usage: 0.00, gen throughput (token/s): 212.85, #queue-req: 0
[2024-11-01 21:16:17] INFO:     127.0.0.1:49030 - "POST /generate HTTP/1.1" 200 OK


# Get Server Args

Used to get the serving args when the server is launched.

In [4]:
# Get server args

url = "http://localhost:30010/get_server_args"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer None"
}

response = requests.get(url, headers=headers)
print_highlight(response.json())

[2024-11-01 21:16:46] INFO:     127.0.0.1:56770 - "GET /get_server_args HTTP/1.1" 200 OK


## Get Model Info

Used to get the model info.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.

In [5]:
url = "http://localhost:30010/get_model_info"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer None"
}

response = requests.get(url, headers=headers)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json["is_generation"] == True
assert response_json.keys() == {"model_path", "is_generation"}

[2024-11-01 21:17:54] INFO:     127.0.0.1:49774 - "GET /get_model_info HTTP/1.1" 200 OK


## Health and Health Generate

- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [6]:
# health generate

url = "http://localhost:30010/health_generate"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer None"
}

response = requests.get(url, headers=headers)
print_highlight(response.text)

# health

url = "http://localhost:30010/health"

response = requests.get(url, headers=headers)
print_highlight(response.text)

[2024-11-01 21:18:13 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 1, cache hit rate: 37.04%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-11-01 21:18:13] INFO:     127.0.0.1:60162 - "GET /health_generate HTTP/1.1" 200 OK


[2024-11-01 21:18:13] INFO:     127.0.0.1:60172 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Used to flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = "http://localhost:30010/flush_cache"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer None"
}

response = requests.post(url, headers=headers)
print_highlight(response.text)

[2024-11-01 21:18:28] INFO:     127.0.0.1:48360 - "POST /flush_cache HTTP/1.1" 200 OK
[2024-11-01 21:18:28 TP0] Cache flushed successfully!


## Get Memory Pool Size

Get the memory pool size in number of tokens.


In [8]:
# get_memory_pool_size

url = "http://localhost:30010/get_memory_pool_size"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer None"
}

response = requests.get(url, headers=headers)
print_highlight(response.text)

[2024-11-01 21:18:42] INFO:     127.0.0.1:37912 - "GET /get_memory_pool_size HTTP/1.1" 200 OK


## Update Weights

Update model weights without restarting the server. Use for continuous evaluation during training. Only applicable for models with the same architecture and parameter size.

In [9]:
# update weights

# successful update with same architecture and size

url = "http://localhost:30010/update_weights"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer None"
}
data = {
    "model_path": "meta-llama/Llama-3.2-1B"
}

response = requests.post(url, headers=headers, json=data)
print_highlight(response.text)
assert response.json()["success"] == True
assert response.json()["message"] == "Succeeded to update model weights."
assert response.json().keys() == {"success", "message"}

[2024-11-01 21:19:26 TP0] Update weights begin. avail mem=1.35 GB
INFO 11-01 21:19:27 weight_utils.py:243] Using model weights format ['*.safetensors']
INFO 11-01 21:19:27 weight_utils.py:288] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.18it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.18it/s]

[2024-11-01 21:19:27 TP0] Update weights end.
[2024-11-01 21:19:27 TP0] Cache flushed successfully!
[2024-11-01 21:19:27] INFO:     127.0.0.1:60486 - "POST /update_weights HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size

url = "http://localhost:30010/update_weights"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer None"
}
data = {
    "model_path": "meta-llama/Llama-3.2-3B"
}

response = requests.post(url, headers=headers, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] == False
assert response_json["message"] == (
    "Failed to update weights: The size of tensor a (2048) must match "
    "the size of tensor b (3072) at non-singleton dimension 1.\n"
    "Rolling back to original weights."
)

[2024-11-01 19:53:00 TP0] Update weights begin. avail mem=1.35 GB
INFO 11-01 19:53:01 weight_utils.py:243] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
INFO 11-01 19:53:01 weight_utils.py:243] Using model weights format ['*.safetensors']
INFO 11-01 19:53:02 weight_utils.py:288] No model.safetensors.index.json found in remote.

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]

Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.65it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.65it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]

[2024-11-01 19:53:02 TP0] Failed to update weights: The size of tensor a (2048) must match the size of tensor b (3072) at non-singleton dimension 1.
Rolling back to original weights.
[2024-11-01 19:53:02] INFO:     127.0.0.1:59090 - "POST /update_weights HTTP/1.1" 40

In [10]:
terminate_process(server_process)